In [2]:
import heapq

# Node class to represent each state
class Node:
    def __init__(self, state, parent=None, cost=0):
        self.state = state
        self.parent = parent
        self.cost = cost

    def __lt__(self, other):
        return self.cost < other.cost

# Expand a node using the problem's action generator
def expand(problem, node):
    children = []
    for neighbor, step_cost in problem['actions'](node.state):
        new_cost = node.cost + step_cost
        children.append(Node(neighbor, parent=node, cost=new_cost))
    return children

# Reconstruct the full path and cost from two meeting nodes
def join_nodes(forward_node, backward_node):
    forward_path = []
    node = forward_node
    while node:
        forward_path.append(node.state)
        node = node.parent
    forward_path.reverse()

    backward_path = []
    node = backward_node
    while node:
        backward_path.append(node.state)
        node = node.parent

    full_path = forward_path + (backward_path[1:] if forward_node.state == backward_node.state else backward_path)

    total_cost = forward_node.cost + backward_node.cost
    return full_path, total_cost

# Termination condition for the bidirectional search
def terminated(solution, frontierF, frontierB, fF, fB):
    if solution is None:
        return False
    if frontierF and frontierB:
        return frontierF[0][0] + frontierB[0][0] >= solution[1]
    return True

# Proceed with expanding one direction of the search
def proceed(direction, problem, frontier, reached, reached_other, solution, heuristic):
   # _, node = heapq.heappop(frontier)
    cost, node = heapq.heappop(frontier) # Just don’t use 'cost' after this

    # Print the node being expanded in this direction
    print(f"{direction} search expanding node: {node.state} with cost: {node.cost}")

    for child in expand(problem, node):
        s = child.state
        if s not in reached or child.cost < reached[s].cost:
            reached[s] = child
            heapq.heappush(frontier, (heuristic(child), child))

            # Print when a node is added to the frontier
            print(f"{direction} search adding node: {s} with cost: {child.cost}")

            if s in reached_other:
                # Meeting point found, try joining
                path, cost = join_nodes(child, reached_other[s]) if direction == 'F' else join_nodes(reached_other[s], child)
                if solution is None or cost < solution[1]:
                    solution = (path, cost)
                    print(f"Meeting point found at: {s}, reconstructing path...")
    return solution

# Main BIBF function
def bibf_search(problemF, fF, problemB, fB):
    nodeF = Node(problemF['initial'])
    nodeB = Node(problemB['initial'])

    frontierF = [(fF(nodeF), nodeF)]
    frontierB = [(fB(nodeB), nodeB)]

    reachedF = {nodeF.state: nodeF}
    reachedB = {nodeB.state: nodeB}

    solution = None

    while not terminated(solution, frontierF, frontierB, fF, fB):
        if frontierF[0][0] <= frontierB[0][0]:
            solution = proceed('F', problemF, frontierF, reachedF, reachedB, solution, fF)
        else:
            solution = proceed('B', problemB, frontierB, reachedB, reachedF, solution, fB)

    return solution

# --------------------------- MAIN PROGRAM --------------------------- #

if __name__ == "__main__":
    graph = {
    'A': [('S', 2), ('C', 2)],
    'S': [('A', 2), ('B', 3)],
    'C': [('A', 2), ('B', 1)],
    'B': [('S', 3), ('C', 1), ('D', 4)],
    'D': [('B', 4)]
}
#     graph = {
#     'A': [('S', 2), ('C', 2), ('D', 7)],
#     'S': [('A', 2), ('B', 5)],
#     'B': [('S', 5), ('D', 1), ('E', 6)],
#     'C': [('A', 2), ('F', 3)],
#     'D': [('A', 7), ('B', 1), ('F', 4), ('G', 2)],
#     'E': [('B', 6), ('G', 1)],
#     'F': [('C', 3), ('D', 4), ('H', 5)],
#     'G': [('D', 2), ('E', 1), ('H', 3), ('I', 2)],
#     'H': [('F', 5), ('G', 3), ('J', 2)],
#     'I': [('G', 2), ('J', 1)],
#     'J': [('H', 2), ('I', 1), ('T', 3)],
#     'T': [('J', 3)]
# }


    def actions(state):
        return graph.get(state, [])

    problemF = {'initial': 'A', 'actions': actions}
    #problemB = {'initial': 'F', 'actions': actions}
    problemB = {'initial': 'D', 'actions': actions}

    if problemF['initial'] not in graph or problemB['initial'] not in graph:
      print("Start or goal state not in graph.")
      exit()


    # Uniform-cost heuristic
    def heuristic(node):
        return node.cost

    solution = bibf_search(problemF, heuristic, problemB, heuristic)

    if solution:
        path, cost = solution
        print("Path found:", " -> ".join(path))
        print("Total cost:", cost)
    else:
        print("No solution found.")


F search expanding node: A with cost: 0
F search adding node: S with cost: 2
F search adding node: C with cost: 2
B search expanding node: D with cost: 0
B search adding node: B with cost: 4
F search expanding node: S with cost: 2
F search adding node: B with cost: 5
Meeting point found at: B, reconstructing path...
F search expanding node: C with cost: 2
F search adding node: B with cost: 3
Meeting point found at: B, reconstructing path...
Path found: A -> C -> B -> D
Total cost: 7
